In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

import gtsam
from gtsam import imuBias, noiseModel, PriorFactorConstantBias
from gtsam.symbol_shorthand import B, V, X, L

from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(g)
# I = np.eye(3)
# IMU_PARAMS.setAccelerometerCovariance(I * 0.2)
# IMU_PARAMS.setGyroscopeCovariance(I * 0.2)
# IMU_PARAMS.setIntegrationCovariance(I * 0.2)
gyro_sigma = 1e-3
accel_sigma = 1e-3
I_3x3 = np.eye(3)
IMU_PARAMS.setGyroscopeCovariance(gyro_sigma**2 * I_3x3)
IMU_PARAMS.setAccelerometerCovariance(accel_sigma**2 * I_3x3)
IMU_PARAMS.setIntegrationCovariance(1e-7**2 * I_3x3)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

pose_noise = gtsam.noiseModel.Diagonal.Sigmas(0.2 * np.ones(6))
# graph.push_back(gtsam.PriorFactorPose3((X(0), gtsam.Pose3(poses[0])), pose_noise))
# Fix first pose
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[0])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[0]))

# IMU prior
BIAS_KEY = B(0)
zero_bias = imuBias.ConstantBias(np.zeros(3), np.zeros(3))
graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, noiseModel.Constrained.All(6)))
initial_estimate.insert(BIAS_KEY, zero_bias)

velocity_key = V(0)
velocity_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.2)
velocity_0 = np.array([0.0, 0.0, 0])
graph.push_back(gtsam.PriorFactorVector(velocity_key, velocity_0, velocity_noise))
initial_estimate.insert(velocity_key, velocity_0)

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

n_frames = 1000

for i in range(1, n_frames):
    accum.integrateMeasurement(imu_data[i, :3], imu_data[i, 3:], DT)

    # Initialize with noisy ground truth poses
    # initial_estimate.insert(pose_key, gtsam.Pose3(poses[i]).compose(DELTA))
    initial_estimate.insert(X(i), gtsam.Pose3(poses[i]))
    initial_estimate.insert(V(i), np.array([0.0, 0.0, 0]))

    graph.add(gtsam.ImuFactor(X(i - 1), V(i - 1), X(i), V(i), BIAS_KEY, accum))

    accum.resetIntegration()

In [ ]:
params = gtsam.LevenbergMarquardtParams()

optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(n_frames)])

fig = plot_poses(poses[:n_frames], no_axes=True, color="black", name="Ground Truth")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM IMU")
fig.show()